# Time deltas solution

Current approach :

For each time delta, data is stored in a large numpy matrix of size [t_delta_Frames x num_MMSI], 2 versions for calculating the position at each frame :
1. Value_at_timestamp on each MMSI for each Frame of the time delta
2. Time precision with Resampling of each MMSI_trajecotry, then simply replacing the corresponding arrays in the matrix

These operations are done in Pymeos within the QGSThread, leading to a big framerate drop, with v2 being slighlty faster but with harder frame drop.



Alternatives that we look at :

. Perfom the temporal precision, resampling AND the indexes in the matrice in the mobilityDB database, this leads to longer times for fetching the matrix and automatically creates a wait time on new t delta

. Instead of keeping a large matrix associated to all time deltas, only keep the tpoints and implement a iterable method


# Current implementation performance

### Using Resampling technique

In [1]:
from pymeos.db.psycopg import MobilityDB
from pymeos import *
from datetime import datetime, timedelta
import time
from collections import deque
from pympler import asizeof
import gc
from enum import Enum
import numpy as np
from shapely.geometry import Point
import math



class Time_granularity(Enum):
    MILLISECOND = {"timedelta" : timedelta(milliseconds=1)}
    SECOND = {"timedelta" : timedelta(seconds=1)}
    MINUTE = {"timedelta" : timedelta(minutes=1)}
    HOUR = {"timedelta" : timedelta(hours=1)}
  


FPS_DEQUEUE_SIZE = 5 # Length of the dequeue to calculate the average FPS
TIME_DELTA_DEQUEUE_SIZE =  10 # Length of the dequeue to keep the keys to keep in the buffer


PERCENTAGE_OF_OBJECTS = 1 # To not overload the memory, we only take a percentage of the ships in the database
TIME_DELTA_SIZE = 240 # Number of frames associated to one Time delta
GRANULARITY = Time_granularity.MINUTE
SRID = 4326
FPS = 20


class Database_connector:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        try: 
            connection_params = {
            "host": "localhost",
            "port": 5432,
            "dbname": "mobilitydb",
            "user": "postgres",
            "password": "postgres"
            }
            self.table_name = "PyMEOS_demo"
            self.id_column_name = "MMSI"
            self.tpoint_column_name = "trajectory"                    
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
            self.ids_list = self.ids_list[:int(len(self.ids_list)*PERCENTAGE_OF_OBJECTS)]
        except Exception as e:
            pass

  
    def get_subset_of_tpoints(self, pstart, pend, xmin, ymin, xmax, ymax):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        try:
           
            ids_list = [ f"'{id[0]}'"  for id in self.ids_list]
            ids_str = ', '.join(map(str, ids_list))
          
            query = f"""
                    SELECT 
                        atStbox(
                            a.{self.tpoint_column_name}::tgeompoint,
                            stbox(
                                ST_MakeEnvelope(
                                    {xmin}, {ymin}, -- xmin, ymin
                                    {xmax}, {ymax}, -- xmax, ymax
                                    4326 -- SRID
                                ),
                                tstzspan('[{pstart}, {pend}]')
                            )
                        )
                    FROM public.{self.table_name} as a 
                    WHERE a.{self.id_column_name} in ({ids_str});
                    """
            self.cursor.execute(query)
            rows = self.cursor.fetchall()
            return rows
        except Exception as e:
            self.log(e)


    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()

In [2]:
PERCENTAGE_OF_OBJECTS = 1
pymeos_initialize()
db = Database_connector()

x_min = -180
y_min = -90
x_max = 180
y_max = 90
start_date = db.get_min_timestamp()
end_date = db.get_max_timestamp()
total_frames = math.ceil( (end_date - start_date) // GRANULARITY.value["timedelta"] )

timestamps = [start_date + i * GRANULARITY.value["timedelta"] for i in range(total_frames)]
timestamps = [dt.replace(tzinfo=None) for dt in timestamps]
timestamps_strings = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in timestamps]
  



In [11]:
# Fetching the tpoints from the database
now = time.time()

begin_frame = 0
end_frame = 240
p_start = timestamps[begin_frame]
p_end = timestamps[end_frame]
rows = db.get_subset_of_tpoints(p_start, p_end, x_min, y_min, x_max, y_max)

TIME_fetch_data = time.time() - now

# Creating the matrix with empty points
now = time.time()

empty_point_wkt = Point().wkt  # "POINT EMPTY"
matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)

TIME_create_matrix = time.time() - now

# Filling the matrix using the resampling technique
now = time.time()

time_ranges = timestamps

for i in range(len(rows)):
    try:
        traj = rows[i][0]
        traj = traj.temporal_precision(GRANULARITY.value["timedelta"]) 
        num_instants = traj.num_instants()
        if num_instants == 0:
            continue
        elif num_instants == 1:
            single_timestamp = traj.timestamps()[0].replace(tzinfo=None)
            index = time_ranges.index(single_timestamp) - begin_frame
            matrix[i][index] = traj.values()[0].wkt
        elif num_instants >= 2:
            traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= GRANULARITY.value["timedelta"])
            
            start_index = time_ranges.index( traj_resampled.start_timestamp().replace(tzinfo=None) ) - begin_frame
            end_index = time_ranges.index( traj_resampled.end_timestamp().replace(tzinfo=None) ) - begin_frame
    
            trajectory_array = np.array([point.wkt for point in traj_resampled.values()])
            matrix[i, start_index:end_index+1] = trajectory_array
    except:
        continue

TIME_fill_matrix = time.time() - now

print(f"time to fetch data for {begin_frame} - {end_frame} : {TIME_fetch_data} s, time to create matrix : {TIME_create_matrix} s, time to fill matrix : {TIME_fill_matrix} s")
TIME_total = TIME_fetch_data + TIME_create_matrix + TIME_fill_matrix
print(f"total time per time delta : {TIME_total} s")

time to fetch data for 0 - 240 : 2.975637674331665 s, time to create matrix : 0.04987382888793945 s, time to fill matrix : 11.17744255065918 s
total time per time delta : 14.202954053878784 s


In [12]:
30*TIME_total

426.0886216163635

In [3]:
time_deltas = [(0, 239), (240, 479), (480, 719), (720, 959), (960, 1199), (1200, 1438)] 

In [4]:
times_record_resampling = []
for time_delta in time_deltas:
    # Fetching the tpoints from the database
    now = time.time()

    begin_frame = time_delta[0]
    end_frame = time_delta[1]
    p_start = timestamps[begin_frame]
    p_end = timestamps[end_frame]
    rows = db.get_subset_of_tpoints(p_start, p_end, x_min, y_min, x_max, y_max)

    TIME_fetch_data = time.time() - now

    # Creating the matrix with empty points
    now = time.time()

    empty_point_wkt = Point().wkt  # "POINT EMPTY"
    matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)

    TIME_create_matrix = time.time() - now

    # Filling the matrix using the resampling technique
    now = time.time()
   
    time_ranges = timestamps

    for i in range(len(rows)):
        try:
            traj = rows[i][0]
            traj = traj.temporal_precision(GRANULARITY.value["timedelta"]) 
            num_instants = traj.num_instants()
            if num_instants == 0:
                continue
            elif num_instants == 1:
                single_timestamp = traj.timestamps()[0].replace(tzinfo=None)
                index = time_ranges.index(single_timestamp) - begin_frame
                matrix[i][index] = traj.values()[0].wkt
            elif num_instants >= 2:
                traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= GRANULARITY.value["timedelta"])
                
                start_index = time_ranges.index( traj_resampled.start_timestamp().replace(tzinfo=None) ) - begin_frame
                end_index = time_ranges.index( traj_resampled.end_timestamp().replace(tzinfo=None) ) - begin_frame
        
                trajectory_array = np.array([point.wkt for point in traj_resampled.values()])
                matrix[i, start_index:end_index+1] = trajectory_array
        except:
            continue

    TIME_fill_matrix = time.time() - now

    print(f"time to fetch data for {begin_frame} - {end_frame} : {TIME_fetch_data} s, time to create matrix : {TIME_create_matrix} s, time to fill matrix : {TIME_fill_matrix} s")
    TIME_total = TIME_fetch_data + TIME_create_matrix + TIME_fill_matrix
    print(f"total time per time delta : {TIME_total} s")
    times_record_resampling.append(TIME_total)



time to fetch data for 0 - 239 : 3.2470123767852783 s, time to create matrix : 0.05280256271362305 s, time to fill matrix : 11.813145637512207 s
total time per time delta : 15.112960577011108 s
time to fetch data for 240 - 479 : 3.1140244007110596 s, time to create matrix : 0.057349205017089844 s, time to fill matrix : 12.651024580001831 s
total time per time delta : 15.82239818572998 s
time to fetch data for 480 - 719 : 3.3441922664642334 s, time to create matrix : 0.057309865951538086 s, time to fill matrix : 13.813710451126099 s
total time per time delta : 17.21521258354187 s
time to fetch data for 720 - 959 : 3.48903489112854 s, time to create matrix : 0.049497365951538086 s, time to fill matrix : 14.447538375854492 s
total time per time delta : 17.98607063293457 s
time to fetch data for 960 - 1199 : 3.086995840072632 s, time to create matrix : 0.05292940139770508 s, time to fill matrix : 12.314752340316772 s
total time per time delta : 15.45467758178711 s
time to fetch data for 12

In [5]:
print(f"Average time to fetch data : {sum([time_record for time_record in times_record_resampling])/len(times_record_resampling)} s")

Average time to fetch data : 15.887430946032206 s


In [54]:
# How many frames at 30 FPS needed for the animation
15.88 * 30

476.40000000000003

### Using Value_at_timestamp

In [9]:
# Fetching the tpoints from the database
now = time.time()

begin_frame = 0
end_frame = 240
p_start = timestamps[begin_frame]
p_end = timestamps[end_frame]
rows = db.get_subset_of_tpoints(p_start, p_end, x_min, y_min, x_max, y_max)

TIME_fetch_data = time.time() - now

# Creating the matrix with empty points
now = time.time()

empty_point_wkt = Point().wkt  # "POINT EMPTY"
matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)

TIME_create_matrix = time.time() - now

# Filling the matrix using the resampling technique
now = time.time()

time_ranges = timestamps

for i in range(len(rows)):
        for j in range(TIME_DELTA_SIZE):
            try:
                traj = rows[i][0]
                try:
                    coords = traj.value_at_timestamp(time_ranges[j + begin_frame])
                    matrix[i][j] = coords.wkt
                except:
                    continue
            except:
                continue
            
TIME_fill_matrix = time.time() - now

print(f"time to fetch data for {begin_frame} - {end_frame} : {TIME_fetch_data} s, time to create matrix : {TIME_create_matrix} s, time to fill matrix : {TIME_fill_matrix} s")
TIME_total = TIME_fetch_data + TIME_create_matrix + TIME_fill_matrix
print(f"total time per time delta : {TIME_total} s")


time to fetch data for 0 - 240 : 2.9861526489257812 s, time to create matrix : 0.05749702453613281 s, time to fill matrix : 11.969017267227173 s
total time per time delta : 15.012666940689087 s


In [10]:
30*TIME_total

450.3800082206726

In [6]:
times_record_vat = []
for time_delta in time_deltas:
    # Fetching the tpoints from the database
    now = time.time()

    begin_frame = time_delta[0]
    end_frame = time_delta[1]
    p_start = timestamps[begin_frame]
    p_end = timestamps[end_frame]
    rows = db.get_subset_of_tpoints(p_start, p_end, x_min, y_min, x_max, y_max)

    TIME_fetch_data = time.time() - now

    # Creating the matrix with empty points
    now = time.time()

    empty_point_wkt = Point().wkt  # "POINT EMPTY"
    matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)

    TIME_create_matrix = time.time() - now

    # Filling the matrix using the resampling technique
    now = time.time()
   
    time_ranges = timestamps

    for i in range(len(rows)):
            for j in range(TIME_DELTA_SIZE):
                try:
                    traj = rows[i][0]
                    try:
                        coords = traj.value_at_timestamp(time_ranges[j + begin_frame])
                        matrix[i][j] = coords.wkt
                    except:
                        continue
                except:
                    continue
              
    TIME_fill_matrix = time.time() - now

    print(f"time to fetch data for {begin_frame} - {end_frame} : {TIME_fetch_data} s, time to create matrix : {TIME_create_matrix} s, time to fill matrix : {TIME_fill_matrix} s")
    TIME_total = TIME_fetch_data + TIME_create_matrix + TIME_fill_matrix
    print(f"total time per time delta : {TIME_total} s")
    times_record_vat.append(TIME_total)



time to fetch data for 0 - 239 : 3.1740968227386475 s, time to create matrix : 0.04974675178527832 s, time to fill matrix : 12.176667928695679 s
total time per time delta : 15.400511503219604 s
time to fetch data for 240 - 479 : 3.097400188446045 s, time to create matrix : 0.05050015449523926 s, time to fill matrix : 13.664529085159302 s
total time per time delta : 16.812429428100586 s
time to fetch data for 480 - 719 : 3.3440070152282715 s, time to create matrix : 0.054293155670166016 s, time to fill matrix : 14.504016637802124 s
total time per time delta : 17.90231680870056 s
time to fetch data for 720 - 959 : 3.269920587539673 s, time to create matrix : 0.0589900016784668 s, time to fill matrix : 14.912182092666626 s
total time per time delta : 18.241092681884766 s
time to fetch data for 960 - 1199 : 3.1655845642089844 s, time to create matrix : 0.060967206954956055 s, time to fill matrix : 13.672136068344116 s
total time per time delta : 16.898687839508057 s
time to fetch data for 

In [7]:
f"average time to fetch data : {sum([time_record for time_record in times_record_vat])/len(times_record_vat)} s"

'average time to fetch data : 16.643550713857014 s'

In [8]:
db.close()

In [55]:
# How many frames at 30 FPS needed for the animation
16.64 * 30

499.20000000000005

# Doing the resampling and indexing for matrix inside the Database

In [29]:
from pymeos.db.psycopg import MobilityDB
from pymeos import *
from datetime import datetime, timedelta
import time
from collections import deque
from pympler import asizeof
import gc
from enum import Enum
import numpy as np
from shapely.geometry import Point
import math



class Time_granularity(Enum):
    MILLISECOND = {"timedelta" : timedelta(milliseconds=1)}
    SECOND = {"timedelta" : timedelta(seconds=1)}
    MINUTE = {"timedelta" : timedelta(minutes=1)}
    HOUR = {"timedelta" : timedelta(hours=1)}
  


FPS_DEQUEUE_SIZE = 5 # Length of the dequeue to calculate the average FPS
TIME_DELTA_DEQUEUE_SIZE =  10 # Length of the dequeue to keep the keys to keep in the buffer


PERCENTAGE_OF_OBJECTS = 1 # To not overload the memory, we only take a percentage of the ships in the database
TIME_DELTA_SIZE = 240 # Number of frames associated to one Time delta
GRANULARITY = Time_granularity.MINUTE
SRID = 4326
FPS = 20


class Database_connector_2:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        try: 
            connection_params = {
            "host": "localhost",
            "port": 5432,
            "dbname": "mobilitydb",
            "user": "postgres",
            "password": "postgres"
            }
            self.table_name = "PyMEOS_demo"
            self.id_column_name = "MMSI"
            self.tpoint_column_name = "trajectory"                    
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
            self.ids_list = self.ids_list[:int(len(self.ids_list)*PERCENTAGE_OF_OBJECTS)]
        except Exception as e:
            pass

  
    def get_subset_of_tpoints(self, pstart, pend, xmin, ymin, xmax, ymax):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        try:
           
            ids_list = [ f"'{id[0]}'"  for id in self.ids_list]
            ids_str = ', '.join(map(str, ids_list))
          
            query = f"""
                    -- Resampling trajectories to 1 minute intervals
                    WITH trajectories AS (
                        SELECT 
                            MMSI,
                            atStbox(
                                a.trajectory::tgeompoint,
                                stbox(
                                     ST_MakeEnvelope(
                                    {xmin}, {ymin}, -- xmin, ymin
                                    {xmax}, {ymax}, -- xmax, ymax
                                    4326 -- SRID
                                ),
                                tstzspan('[{pstart}, {pend}]')
                                )
                            ) as traj
                        FROM public.PyMEOS_demo as a 
                        WHERE a.{self.id_column_name} in ({ids_str})),
                    processed_trajectory AS (
                        SELECT 
                            tprecision(traj, INTERVAL '1 minute', startTimestamp(traj)) AS precise_trajectory
                        FROM 
                            trajectories 
                    ),
                        
                    resampled AS 
                            (SELECT 
                                tsample(precise_trajectory, INTERVAL '1 minute', startTimestamp(precise_trajectory))  AS resampled_trajectory
                                FROM 
                                processed_trajectory
                                ),
                        

                        final_values AS (
                    SELECT
                        startTimestamp(resampled_trajectory) as start_timestamp, endTimestamp(resampled_trajectory) as end_timestamp,
                        resampled_trajectory 
                    FROM 
                        resampled
                    ),

                    -- Resampling trajectories to 1 minute intervals
                    minute_intervals AS (
                        SELECT 
                            generate_series(
                                timestamp '{pstart}', 
                                timestamp '{pend}', 
                                interval '1 minute'
                            ) AS ts
                    ),
                    timestamps_with_index AS (
                        SELECT 
                            ts,
                            row_number() OVER (ORDER BY ts) - 1 AS idx  
                        FROM 
                            minute_intervals
                    ),
                        
                    start_end_indices AS (
                        SELECT 
                            MIN(t.idx) AS start_index,
                            MAX(g.idx) AS end_index,
                            p.resampled_trajectory as traj
                        FROM 
                            final_values p
                        JOIN 
                            timestamps_with_index t ON t.ts = p.start_timestamp
                        JOIN 
                            timestamps_with_index g ON g.ts = p.end_timestamp
                        GROUP BY
                        p.resampled_trajectory

                    )

                    SELECT
                        start_index,
                        end_index,
                        traj
                        
                        
                    FROM
                        start_end_indices;


                    """

            self.cursor.execute(query)
            rows = self.cursor.fetchall()
            return rows
        except Exception as e:
            print(f"Error fetching the subset of Tpoints : {e}")


    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()


In [31]:
PERCENTAGE_OF_OBJECTS = 1
pymeos_initialize()
db = Database_connector_2()

x_min = -180
y_min = -90
x_max = 180
y_max = 90
start_date = db.get_min_timestamp()
end_date = db.get_max_timestamp()
total_frames = math.ceil( (end_date - start_date) // GRANULARITY.value["timedelta"] )

timestamps = [start_date + i * GRANULARITY.value["timedelta"] for i in range(total_frames)]
timestamps = [dt.replace(tzinfo=None) for dt in timestamps]
timestamps_strings = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in timestamps]

In [46]:
times_record_db_resampling = []
for time_delta in time_deltas:
    # Fetching the tpoints from the database
    now = time.time()

    begin_frame = time_delta[0]
    end_frame = time_delta[1]
    p_start = timestamps[begin_frame]
    p_end = timestamps[end_frame]
    rows = db.get_subset_of_tpoints(p_start, p_end, x_min, y_min, x_max, y_max)

    TIME_fetch_data = time.time() - now

    # Creating the matrix with empty points
    now = time.time()

    empty_point = Point()  # "POINT EMPTY"
    matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point, dtype=object)

    TIME_create_matrix = time.time() - now

    # Filling the matrix using the resampling technique
    now = time.time()

    count= 0
    for i in range(len(rows)):
        try:
            traj = rows[i][2]

            start_index = rows[i][0]
            end_index = rows[i][1]

            # trajectory_array = np.array([point.wkt for point in traj.values()])
            matrix[i, start_index:end_index+1] = traj.values()
            count +=1
        except:
            continue

    TIME_fill_matrix = time.time() - now

    print(f"time to fetch data for {begin_frame} - {end_frame} : {TIME_fetch_data} s, time to create matrix : {TIME_create_matrix} s, time to fill matrix : {TIME_fill_matrix} s")
    TIME_total = TIME_fetch_data + TIME_create_matrix + TIME_fill_matrix
    print(f"total time per time delta : {TIME_total} s")
    times_record_db_resampling.append(TIME_total)
    print(count)




time to fetch data for 0 - 239 : 7.0551979541778564 s, time to create matrix : 0.002498149871826172 s, time to fill matrix : 0.304271936416626 s
total time per time delta : 7.361968040466309 s
150
time to fetch data for 240 - 479 : 18.673765182495117 s, time to create matrix : 0.021819114685058594 s, time to fill matrix : 10.037767171859741 s
total time per time delta : 28.733351469039917 s
3433
time to fetch data for 480 - 719 : 21.429890632629395 s, time to create matrix : 0.12171459197998047 s, time to fill matrix : 9.839786767959595 s
total time per time delta : 31.39139199256897 s
3809
time to fetch data for 720 - 959 : 20.870601415634155 s, time to create matrix : 0.15086674690246582 s, time to fill matrix : 10.00284194946289 s
total time per time delta : 31.02431011199951 s
3981
time to fetch data for 960 - 1199 : 19.214648962020874 s, time to create matrix : 0.1467278003692627 s, time to fill matrix : 8.770018577575684 s
total time per time delta : 28.13139533996582 s
3832
time

In [49]:
times_record_db_resampling = times_record_db_resampling[1:]

In [50]:
f"average time to fetch data : {sum([time_record for time_record in times_record_db_resampling])/len(times_record_db_resampling)} s"

'average time to fetch data : 29.080837726593018 s'

In [53]:
# How many frames at 30 FPS needed for the animation
29.08 * 30

872.4

# Try/except clause vs if statement

Triggering clause

In [60]:
%%timeit
a = [i for i in range(100000)]

idx = 1e7
if idx > len(a) or idx < 0 :
    #some operation
    average = sum(a)/len(a)
else:
    a[idx]

1.74 ms ± 13.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [61]:
%%timeit
a = [i for i in range(100000)]

idx = 1e7
try:
    a[idx]
except:
    average = sum(a)/len(a)

1.74 ms ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Without triggering clause

In [62]:
%%timeit
a = [i for i in range(100000)]

idx = 10
if idx > len(a) or idx < 0 :
    #some operation
    average = sum(a)/len(a)
else:
    a[idx]

1.22 ms ± 61.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [63]:
%%timeit
a = [i for i in range(100000)]

idx = 10
try:
    a[idx]
except:
    average = sum(a)/len(a)

1.19 ms ± 29.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Dictionnary update

In [1]:
from shapely.geometry import Point
import numpy as np


array = np.full(5000, Point(0,0).wkt, dtype=object)
array

array(['POINT (0 0)', 'POINT (0 0)', 'POINT (0 0)', ..., 'POINT (0 0)',
       'POINT (0 0)', 'POINT (0 0)'], dtype=object)

In [2]:
%%timeit

dict1 = {}
for i in range(5000):
    dict1[i] = array[i]


219 µs ± 6.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [3]:
%%timeit

dict2 = dict(zip(range(5000), array))

174 µs ± 4.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
